In [1]:
import rasterio
import numpy as np
import random
import time
from numba import njit
from numba.typed import Dict
import utils

# Calibration

### Inputs & Parameters

In [53]:
date_ini = '1994'
date_fin = '2003'

urban_path_ini = 'Inputs/urban2012_roads.gif'
urban_path_fin = 'Inputs/urban2017_roads.gif'
path_roads = 'Inputs/roads_12_mod.tif'
excluded_areas_path = 'Inputs/excluded.gif'

### Reading inputs and estimating new urbanisation

In [54]:
# Urban Areas
urban_ini = rasterio.open(urban_path_ini).read(1)
urban_fin = rasterio.open(urban_path_fin).read(1)

diff_urb = urban_fin - urban_ini
new_urb_index = np.where(diff_urb==255)
new_urb_index = [(a,b) for a,b in zip(new_urb_index[0],new_urb_index[1])]
new_urbanisation = len(new_urb_index)

# Roads
roads = rasterio.open(path_roads).read(1)

# Raster Profile
raster_profile = rasterio.open(urban_path_ini).profile

### Candidate Cells for Growth

In [55]:
candidate_cells = utils.create_growth_candidate_cells(urban_ini,roads,255,255,5,[],0)

edge_growth_cells = candidate_cells[0]
spread_growth_cells = candidate_cells[1]
road_growth_cells = candidate_cells[2]
spont_growth_cells = candidate_cells[3]

### Assigning new urban cells to each type of growth

In [56]:
new_edge = len(edge_growth_cells) - len(list(set(edge_growth_cells) - set(new_urb_index)))
new_road = len(road_growth_cells) - len(list(set(road_growth_cells) - set(new_urb_index)))
new_spread = len(spread_growth_cells) - len(list(set(spread_growth_cells) - set(new_urb_index)))
new_spont = len(spont_growth_cells) - len(list(set(spont_growth_cells) - set(new_urb_index)))

edge_coef = new_edge/new_urbanisation
road_coef = new_road/new_urbanisation
spread_coef = new_spread/new_urbanisation
spont_coef = new_spont/new_urbanisation

In [57]:
print('Edge_G:', np.round(new_edge/new_urbanisation,2),', ','Road_G:',np.round(new_road/new_urbanisation,2),', ',
      'Spread_G:',np.round(new_spread/new_urbanisation,2),', ','Spont_G:', np.round(new_spont/new_urbanisation,2),
      'New urbanisation:', new_urbanisation)

Edge_G: 0.93 ,  Road_G: 0.0 ,  Spread_G: 0.04 ,  Spont_G: 0.02 New urbanisation: 4236
